# Google maps API call setup

In [162]:
import pandas as pd
import requests
import numpy as np
import time

In [281]:
# Set Google Maps API key here
api_key = 'xxxx'

In [464]:
# API call step 1: Geocode the address to get its latitude and longitude
def geocode_address(address):
    geocoding_url = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {
        'address': address,
        'key': api_key,
    }
    response = requests.get(geocoding_url, params=params)
    if response.status_code == 200:
        geocode_data = response.json()
        if geocode_data['status'] == 'OK':
            location = geocode_data['results'][0]['geometry']['location']
            return location
        else:
            print('Geocoding failed. Status:', geocode_data['status'])
            return None
    else:
        print('Geocoding request failed with status code:', response.status_code)
        return None

# API call step 2: List all places within 1 kilometer (default) of the given address
def list_nearby_places(location, radius=1000, next_page_token = ''):
    places_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    if (next_page_token==''):
        params = {
            'location': f'{location["lat"]},{location["lng"]}',
            'radius': radius,
            'key': api_key
        }
    else:
        params = {
            'location': f'{location["lat"]},{location["lng"]}',
            'radius': radius,
            'key': api_key,
            'pagetoken' : next_page_token
        }
    response = requests.get(places_url, params=params)
    if response.status_code == 200:
        places_data = response.json()
        if places_data['status'] == 'OK':
            return places_data
        else:
            print('Nearby Places request failed. Status:', places_data['status'])
            return []
    else:
        print('Nearby Places request failed with status code:', response.status_code)
        return []

# Convert API response to Dataframe
def json_to_dataframe(data):
    data_list = []
    
    for place_data in data:
        name = place_data['name']
        types = ', '.join(place_data['types'])
        rating = place_data.get('rating', None)
        user_ratings_total = place_data.get('user_ratings_total', None)
        
        data_list.append({
            'name': name,
            'types': types,
            'rating': rating,
            'user_ratings_total': user_ratings_total
        })
    
    df = pd.DataFrame(data_list)
    
    return df

# Function to safely compute the logarithm
def safe_log(x):
    if x is None or x <= 0:
        return np.nan
    return np.log1p(x)

# Get a combined score from user ratings and number of ratings
def calculate_combined_score(df, rating_multiplier, user_ratings_total_multiplier):
    # Check if the input DataFrame contains the necessary columns
    if 'rating' not in df or 'user_ratings_total' not in df:
        raise ValueError("DataFrame must have 'rating' and 'user_ratings_total' columns.")
    
    # Calculate the combined score
    df['combined_score'] = round(df['rating'] * rating_multiplier + df['user_ratings_total'].apply(safe_log) * user_ratings_total_multiplier, 1)
    
    # Sort the DataFrame by the combined_score column
    df = df.sort_values(by='combined_score', ascending=False, ignore_index=True)
    
    return df

# Set data display options
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Weights for overall ratings and number of ratings
rating_multiplier = 0.5
user_ratings_total_multiplier = 0.5

In [466]:
# Get 4 new location around original location
def get_new_locations(location, radius):
    expansion = 0.02 * radius / 500
    locations = []
    for lat_delta in [-expansion, expansion]:
        for lng_delta in [-expansion, expansion]:
            new_location = location.copy()
            new_location['lat'] += lat_delta
            new_location['lng'] += lng_delta
            locations.append(new_location)

    locations.append(location)
    return locations


# Main function
def find_places_nearby(address, radius_by_user):
    location0 = geocode_address(address)
    radius = radius_by_user / 2
    all_five_locations = get_new_locations(location0, radius)
    all_places = pd.DataFrame(columns=['name', 'types', 'rating', 'user_ratings_total'])
    for location in all_five_locations:
        if location:
            nearby_places_full = list_nearby_places(location, radius, '')
            nearby_places = nearby_places_full['results']
            time.sleep(2)
            try:
                nearby_places_full['next_page_token']
                nearby_places_full2 = list_nearby_places(location, radius, nearby_places_full['next_page_token'])
                nearby_places2 = nearby_places_full2['results']
            except:
                nearby_places_full2 = []
                nearby_places2 = []
            time.sleep(2)
            try:
                nearby_places_full2['next_page_token']
                nearby_places_full3 = list_nearby_places(location, radius, nearby_places_full2['next_page_token'])
                nearby_places3 = nearby_places_full3['results']
            except:
                nearby_places_full3 = []
                nearby_places3 = []
            # Convert the API response to DataFrame
            if nearby_places:
                places_df = json_to_dataframe(nearby_places+nearby_places2+nearby_places3)
            # Call the function with your DataFrame and multipliers
            all_places = pd.concat([all_places, places_df], ignore_index=True)
        else:
            print("Location could not be found")
    all_places2 = calculate_combined_score(all_places.dropna(), rating_multiplier, user_ratings_total_multiplier)
    all_places3 = all_places2.rename(columns={'name': 'Places Around : ' + address})
    return all_places3.drop_duplicates()

# Output : Call the main function to find places near an address within a radius

### Visualize the Output

In [465]:
# Provide address and radius in meters
interesting_places_closeby = find_places_nearby('Camden market, London', 500)
print(len(interesting_places_closeby))
interesting_places_closeby[interesting_places_closeby['types'].str.contains("tourist_attraction")]

191


,"Places Around : Camden market, London",types,rating,user_ratings_total,combined_score
0,Queen Mary's Rose Gardens,"tourist_attraction, park, point_of_interest, establishment",4.8,4485.0,6.6
4,Regent's Park Open Air Theatre,"tourist_attraction, point_of_interest, establishment",4.7,2614.0,6.3
24,Jewish Museum London,"tourist_attraction, cafe, museum, store, food, point_of_interest, establishment",4.5,628.0,5.5
45,St John's Lodge Garden,"tourist_attraction, park, point_of_interest, establishment",4.9,187.0,5.1
56,Triton and Dryads Fountain,"tourist_attraction, point_of_interest, establishment",4.7,114.0,4.7
138,BOY AND FROG STATUE,"tourist_attraction, point_of_interest, establishment",3.9,11.0,3.2


In [467]:
interesting_places_closeby = find_places_nearby('SpitalFields market, London', 500)
print(len(interesting_places_closeby))
interesting_places_closeby[interesting_places_closeby['types'].str.contains("tourist_attraction")]

210


,"Places Around : SpitalFields market, London",types,rating,user_ratings_total,combined_score
0,London Bridge,"tourist_attraction, point_of_interest, establishment",4.6,46694.0,7.7
1,Monument to the Great Fire of London,"tourist_attraction, point_of_interest, establishment",4.5,11288.0,6.9
6,Wilton's Music Hall,"tourist_attraction, cafe, bar, point_of_interest, food, establishment",4.8,2012.0,6.2
12,Hackney City Farm,"tourist_attraction, point_of_interest, establishment",4.5,1478.0,5.9
13,Jack The Ripper Museum,"tourist_attraction, museum, point_of_interest, establishment",4.1,1636.0,5.8
47,Christ Church Spitalfields,"tourist_attraction, church, place_of_worship, point_of_interest, establishment",4.5,204.0,4.9
73,St Mary-At-Hill,"church, tourist_attraction, place_of_worship, point_of_interest, establishment",4.5,91.0,4.5
